# Evaluating decoding speed

In [1]:
code: str = "bb(n=72,k=12,d=6)"
noise: str = "depolarize(p=0.02)"

kwargs = dict(
    min_time = 60,
    min_shots = 1000000,  # a couple of them might take hours but usually they are fine
    min_init_time = 0.1,
    min_init_shots = 1,
)

starting_index_bias: float = 0.0

json_filename: str | None = None
force_finished: bool = True  # only plot the figure and do not run experiments

In [2]:
ecr_vec: list[float] = [0.2 * e for e in range(5)] + [0.9, 0.95, 0.98, 1]  # erasure conversion rate
c_vec = [0, 50, 1000]
max_iter_vec = [0, 5]
osd_order_vec = [0, 10]

mwpf_decoder_vec = [f"mwpf(c={c})" for c in c_vec] + [f"mwpf(c={c},pass_circuit=1)" for c in c_vec]
bposd0_decoder_vec = [f"bposd(max_iter={max_iter})" for max_iter in max_iter_vec]
bposdn_decoder_vec = [f"bposd(max_iter={max_iter},osd_order={osd_order},osd_method=osd_e)" for max_iter in max_iter_vec for osd_order in osd_order_vec]
bpuf_decoder_vec = [f"bpuf(max_iter={max_iter})" for max_iter in max_iter_vec]

decoder_vec = mwpf_decoder_vec + bposd0_decoder_vec + bposdn_decoder_vec + bpuf_decoder_vec
print("number of decoders:", len(decoder_vec))

number of decoders: 14


In [3]:
from slugify import slugify

if json_filename is None:
    json_filename = "zdat-speed-" + slugify(code) + ".json"
print(json_filename)

zdat-speed-bb-n-72-k-12-d-6.json


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from qec_lego_bench.hpc.job_store import Job, JobStore
from qec_lego_bench.hpc.submitter import *
from qec_lego_bench.hpc.plotter import *
from qec_lego_bench.cli.decoding_speed import decoding_speed, DecodingSpeedResult

### Define the job list

In [6]:
def evaluation_function(
    decoder: str, ecr: float, no_detectors: bool, verbose: bool = True
) -> DecodingSpeedResult:
    if verbose:
        print(f"decoder: {decoder}, ecr: {ecr}, no_detectors: {no_detectors}", flush=True)
    return decoding_speed(
        decoder=decoder,
        code=code,
        noise=noise,
        noise2=f"erasure_conversion(rate={ecr}"
        + (",no_detectors=1)" if no_detectors else ")"),
        **kwargs,
        no_print=not verbose,
    )


jobs = [
    Job(decoder, ecr, no_detectors)
    for decoder in decoder_vec
    for ecr in ecr_vec
    for no_detectors in [True, False]
]
# evaluation_function(decoder_vec[0])

## The rest of the notebook runs the evaluation

### Define the callback, e.g. plotting the intermediate result and the list of remaining tasks

(I have to put them in the same block as the actual execution, otherwise it won't update in VScode)

In [7]:
import time  # add some sleep to let them work properly in VScode Jupyter notebook


def plotter(executor: JobStore):
    ...
    time.sleep(0.1)

job_store = JobStore(evaluation_function, jobs, result_type=DecodingSpeedResult, filename=json_filename)
job_store.execute(loop_callback=plotter, starting_index_bias=starting_index_bias)